In [1]:
!pip install transformers
!pip install -U "ray[tune]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.9 MB/s eta 0:00:00


# Imports

In [2]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
from matplotlib import pyplot as plt
from ray import tune


# Loading prepared train and test data

In [3]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')
train_data['overview'].fillna('', inplace=True)
test_data['overview'].fillna('', inplace=True)

In [4]:
# 2. Dataset and Dataloader
class RevenueDataset(Dataset):
    def __init__(self, tokenizer, data, device, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = data
        self.original_language_cols = [x for x in data.columns if x.startswith('original_language_')]
        self.genre_cols = [x for x in data.columns if x.startswith('genre_')]
        self.cast_cols = [x for x in data.columns if x.startswith('cast_')]
        self.crew_cols = [x for x in data.columns if x.startswith('crew_')]
        self.device = device

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        inputs = self.tokenizer.encode_plus(row['overview'], add_special_tokens=True, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt').to(self.device)

        original_language = torch.tensor(row[self.original_language_cols].values.astype(float), dtype=torch.float, device=self.device)
        genres = torch.tensor(row[self.genre_cols].values.astype(float), dtype=torch.float, device=self.device)
        cast = torch.tensor(row[self.cast_cols].values.astype(float), dtype=torch.float, device=self.device)
        crew = torch.tensor(row[self.crew_cols].values.astype(float), dtype=torch.float, device=self.device)
        budget = torch.tensor(row['budget_100M'], dtype=torch.float, device=self.device)
        budget_unknown = torch.tensor(row['budget_unknown'], dtype=torch.float, device=self.device)
        revenue = torch.tensor(row['revenue_100M'], dtype=torch.float, device=self.device)

        x = torch.cat((
            inputs["input_ids"].squeeze(),
            inputs["attention_mask"].squeeze(),
            original_language,
            genres,
            cast,
            crew,
            budget.unsqueeze(0),
            budget_unknown.unsqueeze(0)
        ))

        return x, revenue

    def __len__(self):
      return len(self.data)

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
train_dataset = RevenueDataset(tokenizer, train_data, DEVICE)
test_dataset = RevenueDataset(tokenizer, test_data, DEVICE)

In [8]:
NUM_GENRES = len(train_dataset.genre_cols)
NUM_CAST = len(train_dataset.cast_cols)
NUM_CREW = len(train_dataset.crew_cols)
NUM_ORIGINAL_LANGUAGES = len(train_dataset.original_language_cols)

In [9]:
# 3. Model
class RevenuePredictor(nn.Module):
    def __init__(self, bert_embedding_size = 128, original_language_embedding_size = 32, cast_embedding_size = 32, crew_embedding_size = 32, hidden_size = 256):
        super(RevenuePredictor, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")

        # Linear layer for textual embeddings
        self.linear_overview = nn.Linear(self.bert.config.hidden_size, bert_embedding_size)

        # Linear layer for original language embeddings
        self.linear_original_language = nn.Linear(NUM_ORIGINAL_LANGUAGES, original_language_embedding_size)

        # Linear layer for embedding cast
        self.linear_cast = nn.Linear(NUM_CAST, cast_embedding_size)

        # Linear layer for embedding crew
        self.linear_crew = nn.Linear(NUM_CREW, crew_embedding_size)

        # Budget and budget_unknown, and genres
        self.other_features_size = 2 + NUM_GENRES

        self.output_layer = nn.Sequential(
            nn.Linear(bert_embedding_size + original_language_embedding_size + cast_embedding_size + crew_embedding_size + self.other_features_size, hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, input):
        bert_out = self.bert(input_ids=input[:, :256].long(), attention_mask=input[:, 256:512].long())
        overview_embedding = self.linear_overview(bert_out['pooler_output'])

        original_language_embedding = self.linear_original_language(input[:, 512:512+NUM_ORIGINAL_LANGUAGES])
        cast_embedding = self.linear_cast(input[:, 512+NUM_ORIGINAL_LANGUAGES:512+NUM_ORIGINAL_LANGUAGES+NUM_CAST])
        crew_embedding = self.linear_crew(input[:, 512+NUM_ORIGINAL_LANGUAGES+NUM_CAST:512+NUM_ORIGINAL_LANGUAGES+NUM_CAST+NUM_CREW])
        other_features = input[:, 512+NUM_ORIGINAL_LANGUAGES+NUM_CAST+NUM_CREW:]


        return self.output_layer(torch.cat((
            overview_embedding,
            original_language_embedding,
            cast_embedding,
            crew_embedding,
            other_features
        ), dim=1))

In [10]:
def raytune_objective(config):
    batch_size = config['batch_size']

    train_dataloader = DataLoader(config['train_dataset'], batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(config['test_dataset'], batch_size=batch_size, shuffle=True)

    NUM_EPOCHS = 10

    model = RevenuePredictor(
        bert_embedding_size=config['bert_embedding_size'],
        original_language_embedding_size=config['original_language_embedding_size'],
        cast_embedding_size=config['cast_embedding_size'],
        crew_embedding_size=config['crew_embedding_size'],
        hidden_size=config['hidden_size']
    )
    optimizer = AdamW(model.parameters(), lr=config['adamw_lr'])
    loss_fn = nn.MSELoss()
    SCHEDULER_STEPS = NUM_EPOCHS * len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=250, num_training_steps=SCHEDULER_STEPS)
    model.to(DEVICE)

    best_mse = float('inf')
    history = []

    for epoch in range(NUM_EPOCHS):
        print("Started epoch ", epoch + 1)
        model.train()
        for x, y in train_dataloader:
            optimizer.zero_grad()

            predictions = model(x)
            loss = loss_fn(predictions, y.unsqueeze(1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            scheduler.step()

            # loop.set_postfix(mseloss=loss.item())

        # Evaluate on test set
        model.eval()
        preds = []
        actuals = []
        with torch.no_grad():
            for x, y in test_dataloader:
                predictions = model(x)
                preds.extend(predictions.squeeze().tolist())
                actuals.extend(y.tolist())
        mse = loss_fn(torch.tensor(preds).unsqueeze(1), torch.tensor(actuals).unsqueeze(1)).item()
        print("MSE on test set after epoch ", epoch + 1, ": ", mse)
        history.append(mse)
        if mse < best_mse:
            best_mse = mse
    print(history)
    # Plot loss history
    plt.plot(history)
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.title('Loss curve')
    plt.show()
    return {'best_mse': best_mse}

In [11]:
search_space = {
    'batch_size': tune.choice([16, 32, 64]),
    'bert_embedding_size': tune.choice([128, 256]),
    'original_language_embedding_size': tune.choice([32, 64]),
    'cast_embedding_size': tune.choice([32, 64]),
    'crew_embedding_size': tune.choice([32, 64]),
    'hidden_size': tune.choice([128, 256, 512]),
    'adamw_lr': tune.loguniform(1e-4, 1e-3),
    'train_dataset': train_dataset,
    'test_dataset': test_dataset
}

tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(raytune_objective),
        resources={"cpu": 2, "gpu": 1}
    ),
    param_space=search_space, tune_config=tune.TuneConfig(num_samples=5)
)

In [ ]:
tune_results = tuner.fit()

2023-11-30 19:13:31,181	INFO worker.py:1673 -- Started a local Ray instance.
2023-11-30 19:13:34,696	INFO tune.py:220 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-11-30 19:13:34,701	INFO tune.py:595 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+--------------------------------------------------------------------------+
| Configuration for experiment     raytune_objective_2023-11-30_19-13-24   |
+--------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                   |
| Scheduler                        FIFOScheduler                           |
| Number of trials                 5                                       |
+--------------------------------------------------------------------------+

View detailed results here: /root/ray_results/raytune_objective_2023-11-30_19-13-24
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/raytune_objective_2023-11-30_19-13-24`

Trial status: 5 PENDING
Current time: 2023-11-30 19:14:43. Total running time: 1min 6s
Logical resource usage: 0/2 CPUs, 0/1 GPUs
+--------------------------------------------------------------------------------------------------------------------

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]
model.safetensors:   2%|▏         | 10.5M/440M [00:00<00:04, 95.3MB/s]
model.safetensors:   7%|▋         | 31.5M/440M [00:00<00:02, 157MB/s] 
model.safetensors:  14%|█▍        | 62.9M/440M [00:00<00:01, 204MB/s]
model.safetensors:  21%|██▏       | 94.4M/440M [00:00<00:01, 223MB/s]
model.safetensors:  29%|██▊       | 126M/440M [00:00<00:01, 219MB/s] 
model.safetensors:  36%|███▌      | 157M/440M [00:00<00:01, 218MB/s]
model.safetensors:  43%|████▎     | 189M/440M [00:00<00:01, 217MB/s]
model.safetensors:  50%|████▉     | 220M/440M [00:01<00:01, 216MB/s]
model.safetensors:  57%|█████▋    | 252M/440M [00:04<00:07, 24.7MB/s]
model.safetensors:  64%|██████▍   | 283M/440M [00:04<00:04, 34.1MB/s]
model.safetensors:  69%|██████▉   | 304M/440M [00:05<00:03, 41.4MB/s]
model.safetensors:  76%|███████▌  | 336M/440M [00:05<00:01, 56.3MB/s]
model.safetensors:  81%|████████  | 357M/440M [00:05<00:01, 66.6MB/s]
model.safetensors:  86%|█████

(raytune_objective pid=1535) Started epoch  1
Trial status: 1 RUNNING | 4 PENDING
Current time: 2023-11-30 19:17:50. Total running time: 4min 12s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                      status       batch_size     bert_embedding_size     ...ge_embedding_size     cast_embedding_size     crew_embedding_size     hidden_size      adamw_lr |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| raytune_objective_8f7d3_00000   RUNNING              32                     128                       64                      64                      32             256   0.00018593  |
| raytune_objective_8f7d3_00001   PENDING              64  

2023-11-30 21:35:14,416	WARNING util.py:202 -- The `callbacks.on_trial_result` operation took 139.416 s, which may be a performance bottleneck.
2023-11-30 21:35:14,419	WARNING util.py:202 -- The `process_trial_result` operation took 139.419 s, which may be a performance bottleneck.
2023-11-30 21:35:14,421	WARNING util.py:202 -- Processing trial results took 139.421 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2023-11-30 21:35:14,422	WARNING util.py:202 -- The `process_trial_result` operation took 139.422 s, which may be a performance bottleneck.


...

(raytune_objective pid=1535) Started epoch  1


2023-11-30 21:37:44,310	ERROR tune_controller.py:1383 -- Trial task failed for trial raytune_objective_8f7d3_00001
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/worker.py", line 2563, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(OutOfMemoryError): ray::ImplicitFunc.train() (pid=1535, ip=172.28.0.12, actor_id=a8373356ac4ac1ec53274a1b01000000, repr=raytune_objective)
  File "/usr/local/lib/python3.10/dist-packages/ray/tune/trainable/trainable.py", line 342, in train
    raise skipped from


Trial raytune_objective_8f7d3_00001 errored after 0 iterations at 2023-11-30 21:37:44. Total running time: 2hr 24min 6s
Error file: /root/ray_results/raytune_objective_2023-11-30_19-13-24/raytune_objective_8f7d3_00001_1_adamw_lr=0.0002,batch_size=64,bert_embedding_size=256,cast_embedding_size=32,crew_embedding_size=6_2023-11-30_19-13-38/error.txt

Trial raytune_objective_8f7d3_00002 started with configuration:
+---------------------------------------------------------------------+
| Trial raytune_objective_8f7d3_00002 config                          |
+---------------------------------------------------------------------+
| adamw_lr                                                    0.00057 |
| batch_size                                                       32 |
| bert_embedding_size                                             256 |
| cast_embedding_size                                              32 |
| crew_embedding_size                                              32 |
| hidden_

(raytune_objective pid=38101) /usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(raytune_objective pid=38101)   warnings.warn(


(raytune_objective pid=38101) Started epoch  1
Trial status: 1 TERMINATED | 1 ERROR | 1 RUNNING | 2 PENDING
Current time: 2023-11-30 21:40:45. Total running time: 2hr 27min 8s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                      status         batch_size     bert_embedding_size     ...ge_embedding_size     cast_embedding_size     crew_embedding_size     hidden_size      adamw_lr     iter     total time (s)     best_mse |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| raytune_objective_8f7d3_00002   RUNNING                32                     256    